## Load required packages

In [1]:
# python basic packages
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy import stats
import time
import pickle
import pandas as pd
import time
from itertools import product
import json
import ast

import pandas as pd
import seaborn as sns

%load_ext autoreload
%autoreload 2
# our packages
from python_modules.utilities import *
from c_modules.c_utilities import *

## Genetate random problems and store them in instances.xlsx file

In [ ]:
%load_ext autoreload
%autoreload 2
# our packages
from python_modules.utilities import *

maindf = pd.DataFrame()
index = -1
for n in [5]:
    for t in [2, 3]:
        for i in range(0, 30):
            if i < 15:
                instance = generate_random_problem(number_of_jobs = n, number_of_machine = n, range_of_processing_times = [1, t], order_of_jobs = 'cycle')
            else:
                instance = generate_random_problem(number_of_jobs = n, number_of_machine = n, range_of_processing_times = [1, t], order_of_jobs = 'randEven')
            json_instance = str(instance)
            index = index + 1
            df = pd.DataFrame({
                'index': [index],
                'number of jobs and machines': [n],
                'upperbound of processing time': [t],
                'instance': [i],
                'problem': [str(instance)]
            })
            maindf = pd.concat([maindf, df], ignore_index=True)
            maindf.to_excel('instances.xlsx', index=False)

## Update Earliness and tardiness costs for generated instances

In [ ]:
maindf = pd.DataFrame()
dfdata = pd.read_excel('instances.xlsx')
# dfdata = dfdata[dfdata['number of jobs and machines'] == 3]
# dfdata = dfdata[dfdata['upperbound of processing time'] == 3]
# dfdata = dfdata[dfdata['instance'] >= 22]
# dfdata = dfdata[dfdata['instance'].isin([17, 18])]

# print(len(dfdata))

dfdata = dfdata.reset_index(drop=True)
maindf = pd.DataFrame()

for i in range(len(dfdata)):
    
    n = dfdata['number of jobs and machines'][i]
    t = dfdata['upperbound of processing time'][i]
    ind = dfdata['instance'][i]
    instance = ast.literal_eval(dfdata['problem'][i])
    for j in range(1, int(n)+1):
        if j == 1:
            instance["w"][j] = 4
        elif j == n:
            instance["w"][j] = 1
        else:
            instance["w"][j] = 2
        for j2 in range(1, int(n)+1):
            if n == 3:
                instance["E"][(j,j2)] = random.uniform(0, 0.667)
            elif n == 4:
                instance["E"][(j,j2)] = random.uniform(0, 0.5)
            elif n == 5:
                instance["E"][(j,j2)] = random.uniform(0, 0.4)
        perv = 0
        machine = 0
        for j2 in range(1, int(n)+1):
            perv = machine
            machine =  instance['sequence'][j][j2-1]
            if perv > 0:
                instance['aggE'][(j, machine)] =  instance['aggE'][(j, perv)] + instance["E"][(j,machine)]
            else:
                 instance['aggE'][(j, machine)] = instance["E"][(j,machine)]
    df = pd.DataFrame({
        'index': [i],
        'number of jobs and machines': [n],
        'upperbound of processing time': [t],
        'instance': [ind],
        'problem': [str(instance)]
    })
    maindf = pd.concat([maindf, df], ignore_index=True)
maindf.to_excel('instances_tmp.xlsx', index=False)

## Solvers Execution

In [38]:
%load_ext autoreload
%autoreload 2
# our packages
from python_modules.utilities import *

time_limit = 300
numer_of_shots = 1000
alpha = 0.01
lamda = 0.7
epsilon = 0.01
lb = 0

maindf = pd.DataFrame()
dfdata = pd.read_excel('instances.xlsx')
# dfdata = dfdata[dfdata['number of jobs and machines'] == 5]
# dfdata = dfdata[dfdata['upperbound of processing time'] == 3]
# dfdata = dfdata[dfdata['instance'] == 2]
# dfdata = dfdata[dfdata['instance'].isin([15, 18])]


dfdata = dfdata.reset_index(drop=True)
for i in range(0, len(dfdata)):
    
    n = dfdata['number of jobs and machines'][i]
    t = dfdata['upperbound of processing time'][i]
    instance = ast.literal_eval(dfdata['problem'][i])
    instance["d"] = {
    key: round(
        sum(instance["p"][(key2, i)] for key2, i in instance["p"] if key2 == key) * 1.1
            )
            for key in instance["d"]
        }
    print(instance["d"])
    print(instance["w"])
    index_in = dfdata['instance'][i]
    index_ = dfdata['index'][i]


    df = pd.DataFrame()
    df['lamda'] = [lamda]

    df['number of jobs and machines'] = [n]
    df['upperbound of processing time'] = [t]
    df['instance'] = [index_in]
    df['index'] = [index_]

    # # Execute ATC + LP
    # start_time = time.time()
    # ub, makespan = ATC_rule_for_ub(instance)
    # stop_time = time.time() - start_time
    # df['total time: atc_lp'] = [stop_time]
    # df['objective value: atc_lp'] = [ub]
    # df['makespan: atc_lp'] = [makespan]

    # Execute CP_WT + LP
    start_time = time.time()
    # objvalue, solving_time, makespan = solvingJJSbyCP(instance, whichobj='wt', time_limit = 300)
    objvalue, solving_time = solvingJJSbyPyomo_Gurobi(instance, whichobj='wt', time_limit = 300)
    stop_time = time.time() - start_time
    df['total time: cp_lp'] = [stop_time]
    df['solving time: cp_lp'] = [solving_time]
    df['objective value: cp_lp'] = [objvalue]
    # df['makespan: cp_lp'] = [makespan]

    ub = objvalue
    lb = objvalue

    # Execute CP_WET
    # start_time = time.time()
    # objvalue, solving_time, makespan = solvingJJSbyCP(instance, whichobj='wet', time_limit = 300)
    # stop_time = time.time() - start_time
    # df['total time: cp_wet'] = [stop_time]
    # df['solving time: cp_wet'] = [solving_time]
    # df['objective value: cp_wet'] = [objvalue]
    # df['makespan: cp_wet'] = [makespan]

    # Execute CPLEX_WET
    # start_time = time.time()
    # objvalue, solving_time = solvingJJSbyCPLEX(instance, whichobj='wet', time_limit = 300)
    # stop_time = time.time() - start_time
    # df['total time: cplex_wet'] = [stop_time]
    # df['solving time: cplex_wet'] = [solving_time]
    # df['objective value: cplex_wet'] = [objvalue]

    # # Execute JJS QA
    # start_time = time.time()
    # v_penalty = max([ub+0.1, 0.1])*2000
    # objvalueQ, objvalueLp = solvingJJSbyQuantumComputing(instance, penalty = v_penalty, numer_of_shots = 1000, howtosolve='qa')
    # stop_time = time.time() - start_time
    # df['total time: jjs_qa'] = [stop_time]
    # df['objective value: jjs_qa'] = [objvalueQ]
    # df['objective value: jjs_qa_lp'] = [objvalueLp]

    # Execute JJS Hybrid
    # start_time = time.time()
    # v_penalty = max([ub+0.1, 0.1])*2000
    # objvalueQ, objvalueLp = solvingJJSbyQuantumComputing(instance, penalty = v_penalty, numer_of_shots = 1000, howtosolve='hybrid')
    # stop_time = time.time() - start_time
    # df['total time: jjs_hybrid'] = [stop_time]
    # df['objective value: jjs_hybrid'] = [objvalueQ]
    # df['objective value: jjs_hybrid_lp'] = [objvalueLp]

    # Execute QA SSP
    # try:
    #     v_penalty = max([ub+0.1, 0.1])
    #     start_time = time.time()
    #     paths, overlap_results, weights, objective_value, variables, loginfo = generate_all_paths_for_jobs_v2(instance, ub+0.5, lb, lamda, epsilon, v_penalty, time_limit, numer_of_shots, howtosolve='quantum_QA')
    #     stop_time = time.time() - start_time
    #     df['total time: ssp_qa'] = [stop_time]
    #     df['objective value: ssp_qa'] = [objective_value]
    #     df['details: ssp_qa'] = [str(loginfo)]
    # except:
    #     f['total time: ssp_qa'] = [9999]
    #     df['objective value: ssp_qa'] = [9999]
    #     df['details: ssp_qa'] = [str(9999)]
    
    # # Execute Hybrid SSP
    # v_penalty = max([ub+0.1, 0.1])
    # start_time = time.time()
    # paths, weights, objective_value, variables, loginfo, total_solving_time = generate_all_paths_for_jobs_v4(instance, ub+0.1, lb, lamda, epsilon, v_penalty, time_limit, numer_of_shots, howtosolve= 'quantum_SA')
    # stop_time = time.time() - start_time
    # df['total time: ssp_hybrid'] = [stop_time]
    # df['objective value: ssp_hybrid'] = [objective_value]
    # df['details: ssp_hybrid'] = [str(loginfo)]

    max_cost_values = [1.222, 1.000, 0.377, 1.000, 0.880, 0.764, 1.000, 2.639, 2.156, 2.000, 2.610, 0.000, 2.239, 1.000, 1.000, 4.000, 4.502, 2.000, 3.000, 3.000, 2.000, 3.246, 4.070, 2.000, 3.000, 2.535, 2.756, 2.273, 3.000, 6.000, 2.112, 3.000, 3.506, 6.000, 4.000, 2.000, 0.710, 0.302, 2.219, 2.000, 0.164, 1.000, 1.000, 2.992, 5.000, 4.000, 4.000, 4.000, 2.000, 2.012, 4.465, 2.000, 5.000, 3.000, 1.300, 1.906, 4.383, 2.659, 4.000, 2.402, 0.744, 4.507, 1.000, 2.274, 0.760, 2.000, 3.654, 1.302, 4.383, 1.128, 2.000, 2.526, 2.783, 2.000, 1.000, 6.212, 8.492, 5.000, 4.246, 8.000, 4.000, 3.000, 2.000, 4.146, 6.000, 4.000, 8.000, 6.165, 6.000, 6.475, 5.200, 6.000, 8.000, 2.000, 3.369, 4.000, 8.000, 6.000, 3.598, 5.000, 3.698, 5.000, 6.022, 3.358, 4.000, 4.000, 7.000, 5.000, 6.492, 5.442, 7.000, 6.000, 14.000, 4.000, 5.000, 5.083, 6.000, 4.560, 8.117, 8.000, 2.141, 1.085, 5.000, 4.000, 2.976, 4.589, 2.098, 4.348, 3.726, 4.000, 4.353, 4.789, 4.599, 7.000, 4.024, 3.000, 8.000, 8.170, 4.626, 10.236, 10.052, 7.332, 5.444, 4.918, 6.330, 3.407, 4.412, 8.000, 5.000, 6.000, 4.642, 3.846, 8.018, 4.628, 7.268, 7.514, 8.000, 6.778, 6.000, 6.000, 6.000, 4.143, 6.620, 7.268, 4.000, 8.467, 8.000, 5.265, 6.380, 12.508, 9.068, 6.126, 7.102, 8.455, 10.153, 9.484, 6.848, 6.677, 8.000, 7.000]
    # Execute Digital SSP
    v_penalty = max([ub+0.5, 1.1])
    start_time = time.time()
    paths, weights, objective_value, variables, loginfo, total_solving_time, n_path_max_before_pruning = generate_all_paths_for_jobs_v4(instance, ub+0.1, lb+0.1, lamda, epsilon, v_penalty, time_limit, numer_of_shots, howtosolve='digital', max_cost_of_optimal = max_cost_values[i], index_name=index_, instance_name=index_in, num_times_name=t)
    stop_time = time.time() - start_time
    df['total time: ssp_digital v1'] = [stop_time]
    df['solving time: ssp_digital v1'] = [total_solving_time]
    df['objective value: ssp_digital v1'] = [objective_value]
    df['details: ssp_digital v1'] = [str(loginfo)]
    
    total_costs_list = [paths[key]["total_costs"] for key in variables if key in paths]
    max_total_costs_list = max(total_costs_list)
    
    df['max_total_costs_list'] = [max_total_costs_list]
    count_number_of_path_smaller_than_max = 0
    for key, path in paths.items():
        if path["total_costs"] <= max_total_costs_list+0.001:
            count_number_of_path_smaller_than_max = count_number_of_path_smaller_than_max + 1
    df['count_number_of_path_smaller_than_max'] = [count_number_of_path_smaller_than_max]
    df['n_path_max_before_pruning'] = [n_path_max_before_pruning]
    df['len_paths'] = [len(paths)]
    df['total_costs_list'] = [total_costs_list]
    
    maindf = pd.concat([maindf, df], ignore_index=True)
    maindf.to_excel('digital_v1.xlsx', index=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{1: 13, 2: 12, 3: 11, 4: 12, 5: 9}
{1: 4, 2: 2, 3: 2, 4: 2, 5: 1}
solveddddddddddddddddddddddd,
solveddddddddddddddddddddddd,
n_overlaps_after 77054
[105, 138, 4568, 1019, 1983]
Iter:  0  Objective value is: 14.41800000704825 lower bound is: 16.54516124128971 upper bound is 16.54516124128971
